# Introduction

본 실습에서는 Bayesian Neural Network(BNN)와 Variational Inference (VI)의 개념을 복습하고, 이를 deep learning에서 구현하는 방법인 Monte-Carlo DropOut (MCDO)의 구현을 알아봅니다. 또한, MCDO를 Dense layer가 아닌 CNN, RNN 등의 구조에서 적용하는 방법에 대해서도 알아봅니다. 

# Recap : Bayesian NN (BNN) and Variational Inference (VI)

일반적인 deep learning의 학습은 다음과 같이 정의됩니다.

* 네트워크 구조 정의 및 파라미터 초기화
* Loss function 정의
* Loss function을 최소화 시키는 파라미터 찾기(with SGD)

이러한 학습 과정은 Loss function을 최소화 시키는 하나의 파라미터 값(여기서 하나의 값은 네트워크를 정의하기 위해 필요한 모든 네트워크 파라미터 값들을 의미합니다.)을 찾기 때문에 빈도주의적 접근(Frequentist approach)라고 표현합니다. 그렇다면 이 과정을 어떻게 사후 분포를 찾는 Bayesian approach로 변화시킬 수 있을까요? 이를 위해서는 먼저 파라미터의 분포 ($p(\theta)$)를 정의하고 위 과정을 다음과 같이 수정하는 방식으로 달성할 수 있습니다.


* 네트워크 구조 정의 및 파라미터의 사전 분포 ($p(\theta)$) 정의
* Likelihood function ($p(\mathcal{D}|\theta)$)정의
* Bayes Theorem($p(\theta| \mathcal{D}) = \frac{p(\theta,\mathcal{D})}{p(\mathcal{D})}$)을 적용하여 파라미터의 사후분포 찾기

이 과정에서 Bayes Theorem은 다음과 같은 적분을 필요로 하며 이를 계산하는 것은 매우 어렵습니다.

$$
p(\mathcal{D}) = \int_{\Omega} p(\mathcal{D} | \theta) p(\theta) d\theta
$$

따라서 이 계산을 우회하기 위해 우리는 파라미터의 사후 분포 ($p(\theta | \mathcal{D}$)를 다음과 같이 해석할 수 있습니다.

$$
p(\theta | \mathcal{D} ) = \arg \inf_{q} \Big\{ -\int_{\Omega} p(\mathcal{D} | \theta ) q(\theta)d\theta + D_{KL}(q(\theta)\| p(\theta)) \Big\}
$$

이때 

$\arg \inf_{q} \Big\{-\int_{\Omega} p(\mathcal{D} | \theta ) q(\theta)d\theta \Big\}$는 제안된 분포 $q(\theta)$가 얼마나 likelihood function $p(\mathcal{D} | \theta )$을 최대화 시키는지를 나타내며,

$\arg \inf_{q} \Big\{D_{KL}(q(\theta)\| p(\theta)) \Big\}$는 제안된 분포 $q(\theta)$가 얼마나 prior $p(\theta)$에 가까운지를 나타냅니다.

즉, 우리가 나타내고자 하는 사후 분포 $p(\theta | \mathcal{D})$는 데이터에 대한 설명력을 높이면서 동시에 사전 분포에 가깝게 하는 문제를 **임의의 파라미터 분포**에 대해서 찾은 답을 의미합니다.

한편 **임의의 파라미터 분포**는 우리가 최적화하기 어려운 대상이므로 이를 최적화하기 쉬운 좀더 작은 분포로 나타낼 수 있습니다. 예를 들어, 파라미터의 분포를 

* 각 파라미터가 서로 독립적이고,
* 각 파라미터가 Gaussian 분포를 따른다고

가정하게 되면 해당 분포는 **각 Gaussian 분포의 평균과 표준편차**만으로 표현할 수 있게 됩니다. 그리고 이러한 분포들 중에서 위 목적 함수를 최대화하는 분포는 각 Gaussian의 평균과 표준편차를 최적화(with SGD)함으로써 찾을 수 있습니다. 이렇게 **임의의 파라미터 분포**에서 최적화하는 사후 분포와 달리, 변분 추론(variational inference, VI)는 우리가 **최적화하기 적합한 더 작은 분포들**에 대해서 최적화하여 근사적으로 사후 분포를 찾는 방법이라고 할 수 있습니다. 정리하면 BNN의 VI는 다음과 같이 적용할 수 있습니다.

* 네트워크 구조 정의 및 파라미터의 사전 분포 ($p(\theta)$) 정의
* Likelihood function ($p(\mathcal{D}|\theta)$)정의
* 최적화 하고자 하는 분포의 범위(e.g. 서로 독립적인 Gaussian) 정의
* Negative likelihood + Prior의 KL divergence가 최소화 되는 파라미터 찾기(with SGD)

앞으로 설명하게 될 MCDO는 최적화 하고자 하는 분포의 범위를 **DropOut으로 표현 가능한 모든 파라미터의 분포**라고 정의합니다.

# Recap : DropOut and Monte-Carlo DropOut (MCDO)

기존의 DropOut은 다음과 같이 train/test time의 네트워크를 다르게 activate 시킵니다.

* Train : 각 레이어의 input을 정해진 확률 $p$만큼 random으로 drop하여 0으로 만들고 계산.("thinned" networks)
* Test : 각 레이어의 모든 input을 계산하되, weight matrix에 $p$만큼 곱하여 evaluation.

![](./images/dropout.png)

위 방법은 frequentist approach에 따라 하나의 파라미터를 찾는 것이라고 볼 수 있습니다. 한편, 사후 분포를 근사적으로 표현하고자 하는 [MCDO 방법론](https://arxiv.org/pdf/1506.02142.pdf)에서 네트워크 파라미터들은 사후 분포를 결정하는 베르누이 분포의 파라미터입니다. 즉, 하나의 예측을 하는 기존의 DropOut 방법과 달리 MCDO에서는 test를 위해 evaluation할 때도 네트워크에 DropOut을 사용합니다. 이때 매번의 DropOut은 **서로 thinned network를 구성하기 때문에 서로 다른 예측**을 만듭니다. 이때,

* 서로 다른 예측들의 평균을 여러 모델들의 평균적인 예측으로 사용하며
* 서로 다른 예측들의 표준편차는 예측의 불확실성으로 사용합니다.

이를 그림으로 나타내면 다음과 같습니다.

![](./images/mcdo.jpg)

* 검은 점 : 모델이 학습에 사용한 데이터(2차원 좌표)
* 검은 실선 : 한 번의 DropOut으로 예측한 함수(thinned network의 prediction)
* 파란 실선 : 여러 번의 DropOut을 통해 얻은 함수값들의 평균
* 파란 영역 : 각 x 좌표별로 y 값의 confidence interval

이후 구현에서는 서로 다른 예측들의 평균을 사용하여 MINST를 예측하는 모델을 구현하도록 하겠습니다.

# MCDO on FFNN

In [1]:
# The network strucuture of this tutorial is based on 
# https://github.com/jwlee-ml/Tensorflow_FastCampus_9th

# import tf and set random seed
import tensorflow as tf
print(tf.test.is_gpu_available())
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(2019)

True


In [2]:
class MLP:
    
    def __init__(self, n_hidden=512, n_output=10):
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        # define placeholders for training MLP
        self.x = tf.placeholder(tf.float32, shape=[None, 784])
        self.y = tf.placeholder(tf.float32, [None, 10])
        self.dropout_rate = tf.placeholder(tf.float32)
        self.lr = tf.placeholder(tf.float32)
        
        self.logit, self.loss, self.opt = self.build_graph(n_hidden=self.n_hidden, n_output=self.n_output)

    def build_graph(self, n_hidden=512, n_output=10):
        """
        Build computational graph for MNIST training with DropOut.

        Args:
            x : MINST input data. (m, 784)
            n_hidden : the number of hidden units in MLP.
            n_output : the size of output layer (=10)
            rate: dropout rate for dropout layers in MLP (tf.placeholder)

        Returns:
            logit : log-probability of prediction. (m, 10)
            loss : cross entropy loss 
            learning rate : learning rate of Adam
        """

        with tf.variable_scope('mlp'):
            # initializers for weight and bias
            w_init = tf.contrib.layers.variance_scaling_initializer()
            b_init = tf.constant_initializer(0.)

            # 1st hidden layer
            w0 = tf.get_variable('w0', [self.x.get_shape()[1], n_hidden], initializer=w_init)
            b0 = tf.get_variable('b0', [n_hidden], initializer=b_init)
            h0 = tf.matmul(self.x, w0) + b0
            h0 = tf.nn.relu(h0)
            h0 = tf.nn.dropout(h0, rate=self.dropout_rate)

            # 2nd hidden layer
            w1 = tf.get_variable('w1', [h0.get_shape()[1], n_hidden], initializer=w_init)
            b1 = tf.get_variable('b1', [n_hidden], initializer=b_init)
            h1 = tf.matmul(h0, w1) + b1
            h1 = tf.nn.relu(h1)
            h1 = tf.nn.dropout(h1, rate=self.dropout_rate)

            # 3nd hidden layer
            w2 = tf.get_variable('w2', [h1.get_shape()[1], n_hidden], initializer=w_init)
            b2 = tf.get_variable('b2', [n_hidden], initializer=b_init)
            h2 = tf.matmul(h1, w2) + b2
            h2 = tf.nn.relu(h2)
            h2 = tf.nn.dropout(h2, rate=self.dropout_rate)

            # 4nd hidden layer
            w3 = tf.get_variable('w3', [h2.get_shape()[1], n_hidden], initializer=w_init)
            b3 = tf.get_variable('b3', [n_hidden], initializer=b_init)
            h3 = tf.matmul(h2, w3) + b3
            h3 = tf.nn.relu(h3)
            h3 = tf.nn.dropout(h3, rate=self.dropout_rate)

            # output layer
            wo = tf.get_variable('wo', [h3.get_shape()[1], n_output], initializer=w_init)
            bo = tf.get_variable('bo', [n_output], initializer=b_init)
            logit = tf.matmul(h3, wo) + bo
            prob = tf.nn.softmax(logit, axis=1)
            
            # loss
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logit, labels=self.y))
            
            # optimizer
            opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(loss)

        return prob, loss, opt

In [3]:
mlp = MLP()
sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth =True)))
sess.run(tf.global_variables_initializer())
# get MINST data
mnist = input_data.read_data_sets("./data/", one_hot=True)

W0716 16:16:20.506014 140433460250432 deprecation.py:323] From <ipython-input-3-26e85f4d7030>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0716 16:16:20.507742 140433460250432 deprecation.py:323] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0716 16:16:20.508680 140433460250432 deprecation.py:323] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future 

Extracting ./data/train-images-idx3-ubyte.gz


W0716 16:16:20.734400 140433460250432 deprecation.py:323] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0716 16:16:20.736351 140433460250432 deprecation.py:323] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0716 16:16:20.778610 140433460250432 deprecation.py:323] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is

Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [4]:
def train_MNIST(net, sess, num_epoch=15, batch_size=100, data=mnist):
        
    sess.run(tf.global_variables_initializer())

    # iterating epoch
    for epoch in range(num_epoch):
        avg_loss = 0.
        num_batch = int(data.train.num_examples / batch_size)

        for i in range(num_batch):
            # get minibatch of X,Y
            batch_xs, batch_ys = data.train.next_batch(batch_size)
            feed_dict = {net.x: batch_xs,
                         net.y: batch_ys,
                         net.dropout_rate: 0.3, 
                         net.lr : 1e-3}
            l, _ = sess.run([net.loss, net.opt], feed_dict=feed_dict)
            avg_loss += l / num_batch

        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_loss))

    print('Learning Finished!')

In [5]:
# evaluation
def evaluate_MNIST(net, sess, data=mnist):
    
    correct_prediction = tf.equal(tf.argmax(net.logit, 1), tf.argmax(net.y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print('DropOut Accuracy:', sess.run(accuracy, feed_dict={net.x: data.test.images,
                                                             net.y: data.test.labels,
                                                             net.dropout_rate: 0.0}))
    
    logits = list()
    for i in range(30):
        logits.append(sess.run(net.logit, feed_dict={net.x: data.test.images,
                                                     net.y: data.test.labels,
                                                     net.dropout_rate: 0.3}))
    logit = np.mean(np.array(logits), 0)
    correct_prediction = np.equal(np.argmax(logit, 1), np.argmax(data.test.labels, 1))
    accuracy = np.mean(correct_prediction)
    print('MCDO Accuracy:', accuracy)

In [6]:
train_MNIST(mlp, sess)

Epoch: 0001 cost = 0.359592794
Epoch: 0002 cost = 0.160491957
Epoch: 0003 cost = 0.122982432
Epoch: 0004 cost = 0.102014680
Epoch: 0005 cost = 0.091578941
Epoch: 0006 cost = 0.079048142
Epoch: 0007 cost = 0.073360531
Epoch: 0008 cost = 0.069570033
Epoch: 0009 cost = 0.061058775
Epoch: 0010 cost = 0.060997845
Epoch: 0011 cost = 0.054926749
Epoch: 0012 cost = 0.052656537
Epoch: 0013 cost = 0.049662874
Epoch: 0014 cost = 0.047724045
Epoch: 0015 cost = 0.046539825
Learning Finished!


In [7]:
evaluate_MNIST(mlp, sess)

DropOut Accuracy: 0.9824
MCDO Accuracy: 0.9826


# MCDO on CNN

MLP에서는 사후분포를 표현하기 위해 기존의 DropOut과 동일하게 **input을 masking하는 방식**으로 MCDO를 구현했습니다.

[MCDO를 CNN에 적용한 연구](https://arxiv.org/pdf/1506.02158.pdf)에서는 이와는 조금 다르게 **convolution 계산 이후 - maxpooling 이전**의 단계에서 masking하는 방식으로 MCDO를 구현합니다. 이를 이해하기 위해 LeNet 5 구조를 보면 다음과 같습니다.

![](./images/lenet.png)

기존의 CNN에 DropOut을 적용하는 연구는 **convolution 레이어에는 DropOut을 적용하지 않고 Fully connected 레이어에만 DropOut을 적용하는 방식**으로 구현되어 있는데 비해, 본 연구에서는 위 그림에서 convolution 과 subsampling 연산 사이에 DropOut을 적용하게 되면 MLP에서와 마찬가지로 CNN에서도 MCDO를 사용할 수 있음을 보였습니다.


이렇게 분포를 정의할 경우 우리는 각 kernel-patch에 대해서 서로 다른 확률 변수를 정의하는 것이 됩니다.

In [8]:
class CNN:
    
    def __init__(self, kernel_size=3, n_hidden=512, n_output=10):
        self.kernel_size = kernel_size
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        # define placeholders for training MLP
        self.x = tf.placeholder(tf.float32, shape=[None, 784])
        self.x_img = tf.reshape(self.x, [-1, 28, 28, 1])
        self.y = tf.placeholder(tf.float32, [None, 10])
        self.dropout_rate = tf.placeholder(tf.float32)
        self.lr = tf.placeholder(tf.float32)
        
        self.logit, self.loss, self.opt = self.build_graph(kernel_size=self.kernel_size, n_hidden=self.n_hidden, n_output=self.n_output)

    def build_graph(self, kernel_size=3, n_hidden=512, n_output=10):
        """
        Build computational graph for MNIST training with DropOut.

        Args:
            x : MINST input data. (m, 784)
            n_hidden : the number of hidden units in MLP.
            n_output : the size of output layer (=10)
            rate: dropout rate for dropout layers in MLP (tf.placeholder)

        Returns:
            logit : log-probability of prediction. (m, 10)
            loss : cross entropy loss 
            learning rate : learning rate of Adam
        """

        with tf.variable_scope('cnn'):
            strides = [1, 1, 1, 1]
            # initializers for weight and bias
            w_init = tf.contrib.layers.variance_scaling_initializer()
            b_init = tf.constant_initializer(0.)

            # 1st hidden layer : (m, 28, 28, 1) -> (m, 14, 14, 32)
            w0 = tf.get_variable('w0', [kernel_size, kernel_size, 1, 32], initializer=w_init)
            h0 = tf.nn.conv2d(self.x_img, w0, strides=strides, padding='SAME')
            h0 = tf.nn.dropout(h0, rate=self.dropout_rate)
            h0 = tf.nn.max_pool(h0, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

            # 2nd hidden layer : (m, 14, 14, 32) -> (m, 7, 7, 64)
            w1 = tf.get_variable('w1', [kernel_size, kernel_size, 32, 64], initializer=w_init)
            h1 = tf.nn.conv2d(h0, w1, strides=strides, padding='SAME')
            h1 = tf.nn.dropout(h1, rate=self.dropout_rate)
            h1 = tf.nn.max_pool(h1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

            # 3nd hidden layer : (m, 7, 7, 64) -> (m, 4, 4, 128)
            w2 = tf.get_variable('w2', [kernel_size, kernel_size, 64, 128], initializer=w_init)
            h2 = tf.nn.conv2d(h1, w2, strides=strides, padding='SAME')
            h2 = tf.nn.dropout(h2, rate=self.dropout_rate)
            h2 = tf.nn.max_pool(h2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
            
            # reshape : (m, 4, 4, 128) -> (m, 4 * 4 * 128)
            h2 = tf.reshape(h2, [-1, 4 * 4 * 128])

            # 4nd hidden layer : (m, 4 * 4 * 128) -> (m, n_hidden)
            w3 = tf.get_variable('w3', [h2.get_shape()[1], n_hidden], initializer=w_init)
            b3 = tf.get_variable('b3', [n_hidden], initializer=b_init)
            h3 = tf.matmul(h2, w3) + b3
            h3 = tf.nn.relu(h3)
            h3 = tf.nn.dropout(h3, rate=self.dropout_rate)

            # output layer : (m, n_hidden) -> (m, n_output)
            wo = tf.get_variable('wo', [h3.get_shape()[1], n_output], initializer=w_init)
            bo = tf.get_variable('bo', [n_output], initializer=b_init)
            logit = tf.matmul(h3, wo) + bo
            prob = tf.nn.softmax(logit, axis=1)
            
            # loss
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logit, labels=self.y))
            
            # optimizer
            opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(loss)

        return prob, loss, opt

In [9]:
# cnn = CNN()

In [10]:
# train_MNIST(cnn, sess)

In [11]:
# evaluate_MNIST(cnn, sess)

# MCDO on RNN

RNN에 dropout을 적용하고자 하는 몇 가지 연구가 있었지만, 기존의 연구들에서는 RNN의 **input/output 레이어에 대해서만** 적용하는 것이 좋다고 알려져 있었습니다. 또한 이렇게 적용되는 DropOut은 **각 input에 대해서 서로 다른 masking**을 사용하는 것이 일반적이었습니다.

하지만, [MCDO를 RNN에 적용한 연구](https://arxiv.org/pdf/1512.05287.pdf)에서는 **모든 input에 대해서 동일한 masking을 사용**할 것을 제안하며, 이렇게 masking을 할 경우 sequential한 정보를 고려하는 **hidden state에 대해서도 DropOut을 적용**할 수 있음을 보였습니다. 이를 그림으로 표현하면 다음과 같습니다.

![](./images/vrnn.png)

그리고 위와 같은 DropOut방식은 tensorflow에서는 [tf.contrib.rnn.DropoutWrapper](https://www.tensorflow.org/api_docs/python/tf/nn/rnn_cell/DropoutWrapper)에 구현이 되어있습니다. 이를 사용하기 위해서는 다음과 같이 이미 정의된 rnn cell 객체를 다음과 같이 wrapping하면 됩니다.

```python
rnn_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_hidden)
keep_prob = 1. - self.dropout_rate
var_rnn_cell = tf.contrib.rnn.DropoutWrapper(rnn_cell,
                                             variational_recurrent=True,
                                             dtype=tf.float32,
                                             input_size=self.x_img.get_shape()[1:],
                                             input_keep_prob=keep_prob,
                                             output_keep_prob=keep_prob,
                                             state_keep_prob=keep_prob,)
```

In [12]:
class RNN:
    
    def __init__(self, kernel_size=3, n_hidden=512, n_output=10):
        self.kernel_size = kernel_size
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        # define placeholders for training MLP
        self.x = tf.placeholder(tf.float32, shape=[None, 784])
#         self.x_img = tf.reshape(self.x, [-1, 28, 28])
        self.y = tf.placeholder(tf.float32, [None, 10])
        self.dropout_rate = tf.placeholder(tf.float32)
        self.lr = tf.placeholder(tf.float32)
        
        self.logit, self.loss, self.opt = self.build_graph(kernel_size=self.kernel_size, n_hidden=self.n_hidden, n_output=self.n_output)

    def build_graph(self, kernel_size=3, n_hidden=512, n_output=10):
        """
        Build computational graph for MNIST training with DropOut.

        Args:
            x : MINST input data. (m, 784)
            n_hidden : the number of hidden units in MLP.
            n_output : the size of output layer (=10)
            rate: dropout rate for dropout layers in MLP (tf.placeholder)

        Returns:
            logit : log-probability of prediction. (m, 10)
            loss : cross entropy loss 
            learning rate : learning rate of Adam
        """

        with tf.variable_scope('rnn'):
            # initializers for weight and bias
            w_init = tf.contrib.layers.variance_scaling_initializer()
            b_init = tf.constant_initializer(0.)

            # rnn cell
            rnn_cell = tf.keras.layers.SimpleRNNCell(units=n_hidden)
            keep_prob = 1. - self.dropout_rate
            var_rnn_cell = tf.contrib.rnn.DropoutWrapper(rnn_cell,
                                                         variational_recurrent=True,
                                                         dtype=tf.float32,
                                                         input_size=self.x.get_shape()[1:],
                                                         input_keep_prob=keep_prob,
                                                         output_keep_prob=keep_prob,
                                                         state_keep_prob=keep_prob,)
            outputs, _ = tf.nn.dynamic_rnn(rnn_cell, self.x_img, dtype=tf.float32)

            # output layer
            wo = tf.get_variable('wo', [n_hidden, n_output], initializer=w_init)
            bo = tf.get_variable('bo', [n_output], initializer=b_init)
            last_rnn_output = outputs[:, -1, :]
            logit = tf.matmul(last_rnn_output, wo) + bo
            prob = tf.nn.softmax(logit, axis=1)
            
            # loss
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logit, labels=self.y))
            
            # optimizer
            opt = tf.train.AdamOptimizer(learning_rate=self.lr).minimize(loss)

        return prob, loss, opt

In [13]:
rnn = RNN()

W0716 16:16:40.563772 140433460250432 deprecation.py:506] From /home/sungyub/.local/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0716 16:16:40.586383 140433460250432 deprecation.py:323] From <ipython-input-12-3d508b3c3d22>:48: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [14]:
train_MNIST(rnn, sess)

Epoch: 0001 cost = 0.351901724
Epoch: 0002 cost = 0.196444149
Epoch: 0003 cost = 0.177925016
Epoch: 0004 cost = 0.149688749
Epoch: 0005 cost = 0.144158772
Epoch: 0006 cost = 0.134444293
Epoch: 0007 cost = 0.137743259
Epoch: 0008 cost = 0.129992990
Epoch: 0009 cost = 0.121905825
Epoch: 0010 cost = 0.126208105
Epoch: 0011 cost = 0.126744313
Epoch: 0012 cost = 0.124897663
Epoch: 0013 cost = 0.123992901
Epoch: 0014 cost = 0.129839284
Epoch: 0015 cost = 0.128008839
Learning Finished!


In [15]:
evaluate_MNIST(rnn, sess)

DropOut Accuracy: 0.9644
MCDO Accuracy: 0.9644
